# 00 - Connecting to the Azure OpenAI API to explore prompt engineering

In this lab, we will connect to our Azure OpenAI API for practicing prompt engineering. For each exercise, you'll get some input text and then an expected completion. Your task is to write the prompt to achieve the expected completion.

But first we will run a few test API calls to make sure everything works.

## Setup

First, we need to retrieve values from the `.env` file which we will use to make calls to the Azure OpenAI API.

In [ ]:
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found OpenAI API Base Endpoint: " + os.getenv("OPENAI_API_BASE"))
else: 
    print("OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("OPENAI_API_KEY")
# This version of the API is needed to properly retrieve the list of model deployments.
API_VERSION = "2023-03-15-preview"
RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE")

Next, we'll set the endpoint that we're going to call. This endpoint will return a list of available model deployments.

In [ ]:
url = RESOURCE_ENDPOINT + "/openai/deployments?api-version=" + API_VERSION

print (url)

Above you will see the full URL that we are going to be calling.

## Get the list of available Model Deployments

Now let's call that Azure OpenAI endpoint and see what it returns. We pass the API key in the HTTP header.

In [ ]:
r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

Above, there should now be a JSON formatted list of model deployments. The output will contain one or more sections similar to the following

```
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt35turbo",
      "status": "succeeded",
      "created_at": 1684150536,
      "updated_at": 1684150536,
      "object": "deployment"
    }
```

In the output, you'll see the **Model Name** as the `model` value and the **Deployment Name** as the `id` value. We'll be using an `id` value for the next section.

## Send our first prompt to Azure OpenAI using the API

Next, let's call the Azure OpenAI API with a prompt. To do this, we'll need the `id` of our completion model deployment.

We've already setup this value in the `.env` file, but you should see a matching value in the list that was output above.

In [ ]:
# Grab API Version in the .env file.
API_VERSION = os.getenv("OPENAI_API_VERSION")
DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

As before, we'll construct a URL to call. This time, we'll also be creating a payload.

In [ ]:
url = RESOURCE_ENDPOINT + "/openai/deployments/" + DEPLOYMENT_ID + "/chat/completions?api-version=" + API_VERSION

print(url)

Again, you will see the full URL that we are going to call. This URL will use the specified deployment to call the **chat completions** API.

Next, we will call the Azure OpenAI API using the URL above. Just like last time, we pass the API key in the HTTP header. We also send a JSON formatted body as part of the request which contains the *prompt* that we want to use to get a response from the OpenAI model. In this case, our prompt is "Once upon a time", which should cause the model to complete the prompt by generating a story.

In [ ]:
content = "Once upon a time"

r = requests.post(url, headers={"api-key": API_KEY}, json={"messages":[{"role": "assistant", "content": content}]})

print(json.dumps(r.json(), indent=2))

As before, the result of the API call will be JSON data similar to the below example.

```
{
  "id": "chatcmpl-7wVxE2LNxaY6ZoxcWAmhiyrqaLZgf",
  "object": "chat.completion",
  "created": 1694180212,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": ", there was a magical kingdom called Tildor ... )))"
      }
    }
  ],
  "usage": {
    "completion_tokens": 366,
    "prompt_tokens": 13,
    "total_tokens": 379
  }
}
```

Here's some more information about some of the data contained in that response.

Key | Description
--- | ---
`model` | The model that was used to generate the response to the prompt
`content` | This is the response that was generated by the OpenAI model. Note that the response example above was edited to remove most of the output to make it easier to read
`finish_reason` | This is the reason that the model stopped generating the response. In this case, `stop` indicates the model determined it had fully completed the response without hitting any limits
`completion_tokens` | The number of tokens that were used in generating the response
`prompt_tokens` | The number of tokens that were consumed by the prompt
`total_tokens` | The total number of tokens that were consumed by the request (`prompt_tokens` + `completion_tokens`)

Now to our first exercise!

You can use the following code snippet where you can set your own prompt into the content variable.

We adjusted the `print` command to make the handling a bit easier.

In [ ]:
content = 'Hello World'

multiline_content = 'This is a multiline string. \
It can span multiple lines and contain line breaks.'

response = requests.post(url, headers={"api-key": API_KEY}, json={"messages":[{"role": "assistant", "content": content}]})

data = response.json()

print(data['choices'][0]['message']['content'])

## :question: Exercise 1 - German Translation

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.`

___

## :question: Exercise 2 - Negation

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `I was not enjoying the sun, and then a huge cloud did not come and cover the sky.`

___

## :question: Exercise 3 - Classification

* Exercise: Write a prompt that generates the expected completion
* Input text: `Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing.`
* Expected completion (or similar):  
  `Positive: 0.75`  
  `Neutral: 0.20`  
  `Negative: 0.05`  

___

## :question: Exercise 4 - E-Mail Summarization

* Exercise: Write a prompt that generates the expected completion
* Input text: Use your own email...
* Expected completion (or similar):  
  `Summary: XYZ`  
  `Open Questions: XYZ`  
  `Action Items: XYZ`  

Find example emails in the Microsoft news site: https://news.microsoft.com/2022/01/18/satya-nadella-email-to-employees-bringing-the-joy-and-community-of-gaming-to-everyone/
___

## :question: Exercise 5 - Rewriting

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `She was enjoying the sun, but then a huge cloud came and covered the sky.`

___

## :question: Exercise 6 - Multiple Tasks

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion:

  ```json
  {  
      "translated": "Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.",
      "negated": "I was not enjoying the sun, and no huge cloud came and covered the sky.",
      "third_person": "She was enjoying the sun, but then a huge cloud came and covered the sky."
  }
  ```
___

## :question: Exercise 7 - Data extraction to JSON

* Exercise: Write a prompt that generates the expected completion
* Input text:

  `Hello, my name is Mateo Gomez. I lost my Credit card on August 17th, and I would like to request its cancellation. The last purchase I made was of a Chicken parmigiana dish at Contoso Restaurant, located near the Hollywood Museum, for $40. Below is my personal information for validation`  
  `Profession: Accountant`  
  `Social Security number is 123-45-6789`  
  `Date of birth: 9-9-1989`  
  `Phone number: 949-555-0110`  
  `Personal address: 1234 Hollywood Boulevard Los Angeles CA`  
  `Linked email account: mateo@contosorestaurant.com`  
  `Swift code: CHASUS33XXX`  

* Expected completion:

  ```json
  {  
      "reason": "Lost card",
      "classified_reason": "lost_card",  
      "name": "Mateo Gomez",  
      "ssn": "123-45-6789",  
      "dob": "09/09/1989"    
  }
  ```
___

## :question: Exercise 8 - Fashion product description

* Exercise: Write a prompt that generates the expected completion
* Input text:

  `Season: Winter`  
  `Style: Sweater` 
  `Gender: Female`  
  `Target group: Teenager`  
  `Material: Cotton` 

* Expected completion (or similar):

  `Stay warm and stylish this winter with our cozy cotton sweaters, perfect for the fashion-forward teenager. Refresh your wardrobe with the latest winter styles from our collection.`

___

## :question: Exercise 9 - Write a Blog Post

* Exercise: Write a blog post about a topic of your choice
* Input text: You choose
* Expected completion: a blogpost with hashtages

___

## :question: Exercise 10 - Call Center

* Exercise: Analyze a call center conversation
* Input text:

  `Employee: "Hello, this is Julia Schreider from Contoso Company. How can I help you today?"`  
  `Customer: "Hi, I am Carsten Mueller. I ordered a package 10 days ago, on February 10th, and it was supposed to arrive in maximum 5 business days.I have called three times already and nobody could provide any more information. I want to know where the package is and I want the problem to be solved immediately. This is the worst service I had for a long time!"`  
  `Employee: "I apologize for the inconvenience, Mr. Mueller. I understand your frustration and I'm here to help. Can you please provide me with your order number so I can look into this for you?"`  
  `Customer: "Yes, it's ACZ456789."`  
  `Employee: "Thank you. I'm looking into it now. Can you please hold for a few minutes while I check the status of your package?"`  
  `Customer: "Okay."`  
  `Employee: "Thank you for your patience. I am sorry to inform you that I am unable to find the status of your package. It appears to have left the sending address, but no up-to-date status on the current location. I will further investigate your case and get back to you as soon as possible via phone call. Could you please provide me your contact information?"`  
  `Customer: "Ah not again. Anyway, my phone number is +4911112223344."`  
  `Employee: "I apologize again for the inconvenience. Is there anything else I can help you with today?"`  
  `Customer: "No."`  
  `Employee: "Thank you. Have a great day!"`  

* Expected completion:

```json
{ 
    "classified_reason": "lost_package",
    "resolve_status": "unresolved",
    "call_summary": "Customer ordered package 10 days ago and has not received it yet.",
    "customer_name": "Carsten Mueller", 
    "employee_name": "Julia Schreider", 
    "order_number": "ACZ456789",  
    "customer_contact_nr": "+4911112223344",  
    "new_address": "N/A",
    "sentiment_initial": ["angry", "frustrated"],
    "sentiment_final": ["calm"],
    "satisfaction_score_initial": 0, 
    "satisfaction_score_final": 5, 
    "eta": "N/A",
    "action_item": ["track_package", "inquire_package_status", "contact_customer"]  
}
```


___

## :question: Exercise 11 - Few-shot learning

* Exercise: Write a few-shot learned prompt that classifies a movie summary.
* Data samples:

  `Paul Atreides, a brilliant and gifted young man born into a great destiny beyond his understanding, must travel to the most dangerous planet in the universe to ensure the future of his family and his people. As malevolent forces explode into conflict over the planet's exclusive supply of the most precious resource in existence-a commodity capable of unlocking humanity's greatest potential-only those who can conquer their fear will survive.`  
  `['Action', 'Adventure', 'Science Fiction’]`

  `A botched store robbery places Wonder Woman in a global battle against a powerful and mysterious ancient force that puts her powers in jeopardy.`  
  `['Action', 'Adventure', 'Fantasy']`

  `After the devastating events of Avengers: Infinity War, the universe is in ruins due to the efforts of the Mad Titan, Thanos. With the help of remaining allies, the Avengers must assemble once more in order to undo Thanos' actions and restore order to the universe once and for all, no matter what consequences may be in store.`  
  `['Adventure', 'Science Fiction', 'Action']`

  `A widowed new dad copes with doubts, fears, heartache and dirty diapers as he sets out to raise his daughter on his own. Inspired by a true story.`     
  `['Drama', 'Family', 'Comedy’]`

  `New data:`  
  `Harry, Ron and Hermione walk away from their last year at Hogwarts to find and destroy the remaining Horcruxes, putting an end to Voldemort's bid for immortality. But with Harry's beloved Dumbledore dead and Voldemort's unscrupulous Death Eaters on the loose, the world is more dangerous than ever.`

* Expected completion: Classification of the new data point

___

## :question: Exercise 12 - NL to SQL with Codex

* Exercise: Write a prompt that generates the expected SQL statement
* Table information:
  * Table: customer // Columns: firstname, name, customer_id, address
  * Table: orders // Columns: order_id, customer_id, product_id, product_amount
  * Table: products // Columns: product_id, price, name, description
* Expected completion: a query that returns the top 10 orders and show the customer name

## Summary

In this lab, we used the Azure OpenAI API directly to query information about our instance of the Azure OpenAI service and to send a prompt to an OpenAI model.

## Next Section

📣 [Integrating AI](../02-integrating-ai/README.md)